In [1]:
# References:
# https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines
# https://stackoverflow.com/questions/3838329/how-can-i-check-if-two-segments-intersect

In [2]:
# Practice Code

def getPositive(x):
    if x>0 :
        return x
    else:
        return None
    
# --- test ---

a = 0
a = getPositive(3)
if(a is None):
    print("NONE")
else:
    print(a)

3


In [3]:
# Test Helper Code

class TestHelper:
    def __init__(self):
        self.Reset()
    def Reset(self):
        self.score = 0
        self.total = 0
    def Check(self,testResult,expectedValue):
        self.total += 1
        if (testResult == expectedValue):
            self.score += 1
            print("T" + str(self.total) + ": ...")
        else:
            print("T" + str(self.total) + ": ERROR")
    def PrintResult(self):
        print("Test Result: " + \
              str(self.score) + "/" + str(self.total) + \
              " ("+ str(round((self.score * 100 / self.total), 2)) +"%)")

In [4]:
# Helper Code

def checkRegion(a1,a2,a3,a4):
    leftMin = 0
    leftMax = 0
    rightMin = 0
    rightMax = 0
    
    if min(a1,a2) < min(a3,a4):
        leftMin = min(a1,a2)
        leftMax = max(a1,a2)
        rightMin = min(a3,a4)
        rightMax = max(a3,a4)
    else:
        leftMin = min(a3,a4)
        leftMax = max(a3,a4)
        rightMin = min(a1,a2)
        rightMax = max(a1,a2)
        
    if leftMax<rightMin: 
        return None
    else:
        return max(leftMin,rightMin), min(leftMax,rightMax)

    
def getGradient(x1,y1,x2,y2):
    if(x2-x1 == 0):
        return None
    else:
        return (y2-y1)/(x2-x1)
    
def checkBackward(x1,y1,x2,y2,x3,y3):
    if(getGradient(x1,y1,x2,y2) == getGradient(x2,y2,x3,y3)):
        if(getGradient(x1,y1,x2,y2) is None):
            if((y1-y2)*(y2-y3) < 0):
                return False
            else:
                return True
        else:
            if((x1-x2)*(x2-x3) < 0):
                return False
            else:
                return True
    else:
        return False;

def checkInclusiveBetween1D(low,mid,high):
    return (low-mid)*(mid-high) >= 0

def checkInclusiveBetween2D(x,o,y):
    return (checkInclusiveBetween1D(x[0],o[0],x[1]) and checkInclusiveBetween1D(y[0],o[1],y[1]))

def flip(a,b):
    return b,a

In [5]:
# Helper Code (Stolen)

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None #determinant 0, both lines have same gradient

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

# --- test ---

testA=[1,1]
testB=[3,3]
testC=[3,1]
testD=[1,3]

testNode = 0,0
testNode = line_intersection([testA,testB],[testC,testD])
if testNode is None:
    print("NONE")
else:
    print(testNode)

(2.0, 2.0)


In [6]:
# Core Code

# Definition
# - returns true when lines are intersecting
# - returns false when lines are not touching
# - tip-to-tip touch is considered false
# - tip-to-edge touch is considered true
# Line Equation: y = Mx * C
def checkIntersect(x1,y1,x2,y2,x3,y3,x4,y4):
    rangeX = checkRegion(x1,x2,x3,x4)
    rangeY = checkRegion(y1,y2,y3,y4)
    
    if(rangeX is not None and rangeY is not None):
        
        # --- checking tip-to-tip touch
        if(x1==x3 and y1==y3):
            return not checkBackward(x2,y2,x1,y1,x4,y4)
        if(x1==x4 and y1==y4):
            return not checkBackward(x2,y2,x1,y1,x3,y3)
        if(x2==x3 and y2==y3):
            return not checkBackward(x1,y1,x2,y2,x4,y4)
        if(x2==x4 and y2==y4):
            return not checkBackward(x1,y1,x2,y2,x3,y3)
        
        # --- getting gradient M ---
        line1M = getGradient(x1,y1,x2,y2)
        line2M = getGradient(x3,y3,x4,y4)

        # --- checking 2 vertical lines ---
        if (line1M is None and line2M is None):
            return True #tip-to-tip touch already checked, therefore this must be 2 verticals intersecting at some interval
        # --- checking 2 horizontal lines ---
        if (line1M == 0 and line2M == 0):
            return True #tip-to-tip touch already checked, therefore this must be 2 horizontals intersecting at some interval
        
        # --- prepping nodes and lines (tuples)
        node1 = x1, y1
        node2 = x2, y2
        node3 = x3, y3
        node4 = x4, y4
        line1 = node1, node2
        line2 = node3, node4
        
        # --- checking 1 vertical line ---
        if(line1M is None or line2M is None):
            if(line1M == 0 or line2M == 0):
                return True #tip-to-edge touch between vertical and horizontal line
            else: #flip so that there's no vertical lines before calculation
                line1 = flip(*node1) , flip(*node2)
                line2 = flip(*node3) , flip(*node4)
                iTemp = line_intersection(line1,line2)
                iNode = flip(*iTemp)
                return checkInclusiveBetween2D(rangeX,iNode,rangeY)
        
        # --- normal case (no vertical lines) ---
        iNode = line_intersection(line1,line2)
        if(iNode is None): #parallel lines, M1 == M2
            line1C = y1 - line1M * x1
            line2C = y3 - line2M * x3
            return line1C == line2C #intersecting when colinear, detached when non-colinear
        else: #intersecting at point
            return checkInclusiveBetween2D(rangeX,iNode,rangeY)
    
    else:
        return False #no intersecting region found

In [7]:
# Test Code
testHelper = TestHelper()

# 1 - detached horizontal lines
A=[1,1]
B=[2,1]
C=[3,1]
D=[4,1]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 2 - intersecting perpendicular lines at point
A=[1,1]
B=[3,1]
C=[2,0]
D=[2,2]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 3 - intersecting X lines at point
A=[1,1]
B=[3,3]
C=[3,1]
D=[1,3]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 4 - tip-to-edge touch at point
A=[1,1]
B=[2,2]
C=[3,1]
D=[1,3]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 5 - intersecting horizontal lines at interval
A=[1,1]
B=[3,1]
C=[2,1]
D=[4,1]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 6 - intersecting diagonal lines at interval
A=[1,1]
B=[3,3]
C=[2,2]
D=[4,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 7 - tip-to-tip touch diagonal lines
A=[1,1]
B=[2,2]
C=[3,3]
D=[2,2]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 8 - intersecting diagonal lines at interval
A=[1,1]
B=[3,3]
C=[2,2]
D=[3,3]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 9 - detached diagonal lines
A=[1,1]
B=[2,2]
C=[3,3]
D=[4,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 10 - detached vertical lines
A=[1,1]
B=[1,2]
C=[1,3]
D=[1,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 11 - intersecting vertical lines at interval
A=[1,1]
B=[1,3]
C=[1,2]
D=[1,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 12 - tip-to-tip vertical lines at point
A=[1,1]
B=[1,3]
C=[1,3]
D=[1,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 13 - parallel diagonal lines
A=[1,1]
B=[3,3]
C=[1,2]
D=[3,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 14 - parallel horizontal lines
A=[1,1]
B=[3,1]
C=[1,2]
D=[3,2]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 15 - parallel vertical lines
A=[1,1]
B=[1,3]
C=[2,1]
D=[2,3]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 16 - not-yet tip-to-edge diagonal lines
A=[1,1]
B=[2,2]
C=[1,4]
D=[4,1]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 17 - tip-to-edge diagonal-vertical lines
A=[1,1]
B=[4,4]
C=[2,2]
D=[2,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 18 - not-yet tip-to-edge diagonal-vertical lines
A=[1,1]
B=[4,4]
C=[2,3]
D=[2,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 19 - intersecting diagonal-vertical lines
A=[1,1]
B=[4,4]
C=[2,1]
D=[2,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 20 - tip-to-edge diagonal-horizontal lines
A=[2,2]
B=[4,2]
C=[1,1]
D=[4,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 21 - not-yet tip-to-edge diagonal-horizontal lines
A=[3,2]
B=[4,2]
C=[1,1]
D=[4,4]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 22 - intersecting diagonal-horizontal lines
A=[1,2]
B=[4,2]
C=[1,1]
D=[4,4]
answer = True
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 23 - not-yet tip-to-tip diagonal lines
A=[1,1]
B=[4,4]
C=[3,2]
D=[5,3]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 24 - not-yet intersecting diagonal lines
A=[1,1]
B=[5,3]
C=[1,3]
D=[2,2]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

# 25 - not-yet intersecting diagonal lines
A=[1,1]
B=[5,3]
C=[1,2]
D=[2,3]
answer = False
testHelper.Check(checkIntersect(*A,*B,*C,*D),answer)

testHelper.PrintResult()

T1: ...
T2: ...
T3: ...
T4: ...
T5: ...
T6: ...
T7: ...
T8: ...
T9: ...
T10: ...
T11: ...
T12: ...
T13: ...
T14: ...
T15: ...
T16: ...
T17: ...
T18: ...
T19: ...
T20: ...
T21: ...
T22: ...
T23: ...
T24: ...
T25: ...
Test Result: 25/25 (100.0%)
